# CAPM 3rd Moment Factor Portfolio
## Chapter 5 Skewness of returns

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import warnings

# Constants
R_f = 0.13
FIGURE_SIZE = (10, 6)

# Load market and industry portfolio data from Excel files, dropping the 'Date' column
df_industries = pd.read_excel('./Homework_4/data/Industry_Portfolios.xlsx', index_col='Date')
df_market = pd.read_excel('./Homework_4/data/Market_Portfolio.xlsx', index_col='Date')
df_risk_factors = pd.read_excel('./Homework_4/data/Risk_Factors.xlsx', index_col='Date')
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

def calculate_coskewness(portfolio_returns, market_returns):
    """
    Calculate the coskewness between portfolio returns and market returns.
    
    Parameters:
    portfolio_returns: Series containing portfolio returns
    market_returns: Series containing market returns
    
    Returns:
    float: Coskewness measure
    """
    # Standardize returns
    p_std = (portfolio_returns - portfolio_returns.mean()) / portfolio_returns.std()
    m_std = (market_returns - market_returns.mean()) / market_returns.std()
    
    # Calculate coskewness
    coskewness = np.mean(p_std * m_std**2)
    return coskewness

def ThreeMomentCAPM(data_portfolio, data_market):
    """
    Calculate Alpha, Beta and Gamma (coskewness) for each portfolio using Three-Moment CAPM.
    
    Parameters:
    data_portfolio: DataFrame containing portfolio returns
    data_market: DataFrame containing market returns
    
    Returns:
    DataFrame with Alpha, Beta and Gamma for each portfolio
    """
    # Align market data with portfolio data by index
    data_market = data_market.reindex(data_portfolio.index)
    
    # Calculate excess returns
    excess_market_returns = data_market.squeeze() - R_f
    excess_portfolio_returns = data_portfolio - R_f
    
    # Initialize results DataFrame
    results = pd.DataFrame(index=excess_portfolio_returns.columns, 
                         columns=['Alpha', 'Beta', 'Gamma'])
    
    # Calculate for each portfolio
    for portfolio in results.index:
        y = excess_portfolio_returns[portfolio]
        x = excess_market_returns
        df = pd.concat([x, y], axis=1).dropna()
        
        # Calculate Beta using linear regression
        model = LinearRegression().fit(df.iloc[:, 0].values.reshape(-1, 1), df.iloc[:, 1])
        alpha, beta = model.intercept_, model.coef_[0]
        
        # Calculate Gamma (coskewness)
        gamma = calculate_coskewness(df.iloc[:, 1], df.iloc[:, 0])
        
        results.loc[portfolio] = [alpha, beta, gamma]
    
    # Set market values
    results.loc['Market'] = [0.0, 1.0, 1.0]
    return results.round(6)

def plot_3M_SML(beta_range, gamma_range, sml_surface, capm_data):
    """
    Plot the Three-Moment Security Market Line Surface.
    
    Parameters:
    beta_range: array of Beta values
    gamma_range: array of Gamma values
    sml_surface: 2D array of expected returns
    capm_data: DataFrame containing portfolio data
    """
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    # Create mesh grid for surface plot
    beta_mesh, gamma_mesh = np.meshgrid(beta_range, gamma_range)
    
    # Plot the surface
    surf = ax.plot_surface(beta_mesh, gamma_mesh, sml_surface, 
                          cmap='viridis', alpha=0.6)
    
    # Plot the actual portfolio points
    ax.scatter(capm_data['Beta'], capm_data['Gamma'], capm_data['Mean_Return'],
              color='red', s=50, label='Portfolios')
    
    ax.set_xlabel('Beta')
    ax.set_ylabel('Gamma')
    ax.set_zlabel('Expected Return')
    ax.set_title('Three-Moment CAPM Security Market Surface')
    
    plt.colorbar(surf)
    plt.legend()
    plt.show()

# Calculate Three-Moment CAPM results
three_m_results = ThreeMomentCAPM(df_industries, df_market)
print("\nThree-Moment CAPM Results:")
print(three_m_results)



Three-Moment CAPM Results:
           Alpha      Beta     Gamma
NoDur   0.369443  0.652647 -0.818511
Durbl  -0.415599  1.648536 -0.535135
Manuf   0.159771  1.169846  -0.79132
Enrgy   0.501719   0.96985 -0.619428
HiTec   -0.06402  1.132969 -0.637821
Telcm   0.194691  0.900729 -0.773927
Shops   0.275492  0.826492 -0.639705
Hlth    0.237841  0.673036 -0.707915
Utils   0.444585  0.538086 -0.753663
Other  -0.387135  1.207309  -0.72898
Market       0.0       1.0       1.0


/Users/ju/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/Users/ju/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/Users/ju/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
